In [1]:
# Import necessary libraries
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Configure imports
sys.path.append(str(Path('..').resolve().parent))
from src.config import FUNK_SVD_MODEL_DIR
from src.recommender import GameRecommender
from src.utils.utils import get_train_data, get_test_data


In [2]:
# Load data and initialize recommender
train_data = get_train_data()
test_data = get_test_data()
model_path = FUNK_SVD_MODEL_DIR / 'model' / 'final'
recommender = GameRecommender(model_path, train_data)
recommender.load_games_data()


2025-04-13 22:07:28.936 | INFO     | src.utils.persistence:load_model:42 - Loading model from C:\Ukma\coursework\funk-svd\models\funk_svd\model\final


In [3]:
# Test case 1: Add a new user with some sample ratings
new_user_ratings = [
    {'BggId': 174430, 'Rating': 8.5},  # Gloomhaven
    {'BggId': 167791, 'Rating': 7.0},  # Terraforming Mars
    {'BggId': 233078, 'Rating': 9.0},  # Wingspan
    {'BggId': 169786, 'Rating': 6.5},  # Scythe
]

# Add the new user and get their ID
success, new_user_id, is_new_user = recommender.add_user_ratings(new_user_ratings)
print(f"Added new user with ID: {new_user_id} (New user: {is_new_user})")

# Get recommendations for the new user
attributes = ['Name', 'YearPublished', 'AvgRating']
recommendations = recommender.get_recommendations(
    user_id=new_user_id,
    n=10,
    attributes=attributes
)

# Display recommendations
print("\nRecommendations for new user:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['Name']} ({rec['YearPublished']}) - Predicted Rating: {rec['PredictedRating']:.2f} (BGG Avg: {rec['AvgRating']:.2f})")


2025-04-13 22:07:30.491 | INFO     | src.funk_svd:add_ratings:280 - Generated new user ID: 320498
2025-04-13 22:07:30.493 | INFO     | src.funk_svd:add_ratings:291 - Added new user 320498 with 4 ratings
2025-04-13 22:07:30.495 | DEBUG    | src.funk_svd:_learn_user_factors:252 - Finished learning factors for user with final RMSE: 0.9872


Added new user with ID: 320498 (New user: True)

Recommendations for new user:
1. Sleeping Gods (2021) - Predicted Rating: 8.82 (BGG Avg: 8.51)
2. Eclipse: Second Dawn for the Galaxy (2020) - Predicted Rating: 8.72 (BGG Avg: 8.68)
3. The Castles of Burgundy (2019) - Predicted Rating: 8.63 (BGG Avg: 8.49)
4. Clank!: Legacy – Acquisitions Incorporated (2019) - Predicted Rating: 8.60 (BGG Avg: 8.70)
5. Concordia Venus (2018) - Predicted Rating: 8.59 (BGG Avg: 8.35)
6. The Crew: Mission Deep Sea (2021) - Predicted Rating: 8.57 (BGG Avg: 8.45)
7. Gloomhaven: Jaws of the Lion (2020) - Predicted Rating: 8.54 (BGG Avg: 8.70)
8. Pandemic Legacy: Season 0 (2020) - Predicted Rating: 8.45 (BGG Avg: 8.64)
9. Ticket to Ride: 10th Anniversary (2014) - Predicted Rating: 8.41 (BGG Avg: 8.27)
10. Aeon's End: War Eternal (2017) - Predicted Rating: 8.41 (BGG Avg: 8.39)


In [4]:
# Test case 2: Add new ratings for an existing user
# First, let's select an existing user ID from the dataset
existing_user_id = train_data[0]['UserId']
print(f"Selected existing user ID: {existing_user_id}")

# Get current recommendations for this user
print("\nCurrent recommendations for existing user:")
existing_recommendations = recommender.get_recommendations(
    user_id=existing_user_id,
    n=5,
    attributes=attributes
)
for i, rec in enumerate(existing_recommendations, 1):
    print(f"{i}. {rec['Name']} ({rec['YearPublished']}) - Predicted Rating: {rec['PredictedRating']:.2f}")

# Add new ratings for this existing user
new_ratings_for_existing = [
    {'BggId': 266192, 'Rating': 9.5},  # Wingspan
    {'BggId': 224517, 'Rating': 8.0},  # Brass: Birmingham
    {'BggId': 162886, 'Rating': 7.5},  # Spirit Island
]

# Update the user with new ratings
success, user_id, is_new_user = recommender.add_user_ratings(new_ratings_for_existing, existing_user_id)
print(f"\nAdded new ratings for existing user {user_id} (New user: {is_new_user})")

# Get updated recommendations for this user
print("\nUpdated recommendations for existing user:")
updated_recommendations = recommender.get_recommendations(
    user_id=existing_user_id,
    n=5,
    attributes=attributes
)
for i, rec in enumerate(updated_recommendations, 1):
    print(f"{i}. {rec['Name']} ({rec['YearPublished']}) - Predicted Rating: {rec['PredictedRating']:.2f}")


2025-04-13 22:07:30.582 | INFO     | src.funk_svd:add_ratings:295 - Updating existing user 4 with 3 new ratings
2025-04-13 22:07:30.584 | DEBUG    | src.funk_svd:_learn_user_factors:252 - Finished learning factors for user with final RMSE: 0.9771


Selected existing user ID: 4

Current recommendations for existing user:
1. Sleeping Gods (2021) - Predicted Rating: 9.23
2. Eclipse: Second Dawn for the Galaxy (2020) - Predicted Rating: 9.13
3. The Castles of Burgundy (2019) - Predicted Rating: 9.04
4. Clank!: Legacy – Acquisitions Incorporated (2019) - Predicted Rating: 9.01
5. Concordia Venus (2018) - Predicted Rating: 9.00

Added new ratings for existing user 4 (New user: False)

Updated recommendations for existing user:
1. Sleeping Gods (2021) - Predicted Rating: 9.21
2. Eclipse: Second Dawn for the Galaxy (2020) - Predicted Rating: 9.11
3. The Castles of Burgundy (2019) - Predicted Rating: 9.02
4. Clank!: Legacy – Acquisitions Incorporated (2019) - Predicted Rating: 8.99
5. Concordia Venus (2018) - Predicted Rating: 8.98


In [5]:
# Compare the before and after recommendations
print("\nComparison of recommendations before and after adding new ratings:")
print("Before:")
before_games = {rec['Name']: rec['PredictedRating'] for rec in existing_recommendations}
for name, rating in before_games.items():
    print(f"- {name}: {rating:.2f}")

print("\nAfter:")
after_games = {rec['Name']: rec['PredictedRating'] for rec in updated_recommendations}
for name, rating in after_games.items():
    print(f"- {name}: {rating:.2f}")

# Get popular recommendations for comparison
popular_games = recommender.get_popular_recommendations(train_data, n=5)
print("\nPopular recommendations:")
for i, (game_id, score) in enumerate(popular_games, 1):
    game_info = recommender.games_data.get(int(game_id), {})
    name = game_info.get('Name', f'Game {game_id}')
    print(f"{i}. {name} - Popularity Score: {score:.2f}")



Comparison of recommendations before and after adding new ratings:
Before:
- Sleeping Gods: 9.23
- Eclipse: Second Dawn for the Galaxy: 9.13
- The Castles of Burgundy: 9.04
- Clank!: Legacy – Acquisitions Incorporated: 9.01
- Concordia Venus: 9.00

After:
- Sleeping Gods: 9.21
- Eclipse: Second Dawn for the Galaxy: 9.11
- The Castles of Burgundy: 9.02
- Clank!: Legacy – Acquisitions Incorporated: 8.99
- Concordia Venus: 8.98

Popular recommendations:
1. Terraforming Mars - Popularity Score: 45.22
2. The Castles of Burgundy - Popularity Score: 43.45
3. Pandemic Legacy: Season 1 - Popularity Score: 43.09
4. Scythe - Popularity Score: 42.69
5. Agricola - Popularity Score: 42.09
